# Este caso de estudio servirá para demostrar el uso del paquete AutoGluon que permite llevar a cabo un prototipado rápido de moodelos de Aprendizaje de Máquina (ML)

## IMPORTAR LIBRERIAS Y CONJUNTO DE DATOS

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Para que funcione Autogluon en Google Colab, es necesario instalar ipkernel y reiniciar el Notebook
# El kernel IPython es el backend de ejecución para Jupyter

!pip install -U ipykernel

In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon --no-cache-dir

In [ ]:
# pip install autogluon autogluon.tabular "mxnet<2.0.0"

El AutoGluon esta modularizado en sub-módulos para datos:  
* Tabular, 
* text,
* Images

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
seguros = pd.read_csv('insurance.csv')

In [ ]:
seguros

**PRACTICA #1 [OPCIONAL]:** 
- **¿Cuántas regiones únicas hay en el DataFrame seguros?**

## REALIZAR UN ANALISIS EXPLORATORIO DE DATOS (EDA) 

In [ ]:
# Explorar las primeras 3 filas del DataFrame

seguros.head(3)

In [ ]:
# Explorar las últimas 5 filas del DataFrame

seguros.tail()

In [ ]:
# Generar un resúmen estadístico

seguros.describe()

In [ ]:
# Obtener información del conjunto de datos

seguros.info()

In [ ]:
# Agrupar por región para ver si hay alguna relación entre region y charges
# Parece ser que la región south east tiene los cargos y los índices de masa corporal más altos

df_region = seguros.groupby('region').describe()
df_region

**PRACTICA #2 [OPCIONAL]:**
- **Agrupe los datos por 'age' y examine la relación entre 'age' y 'charges'**

## REALIZAR UNA VISUALIZACION DE LOS DATOS

In [ ]:
# Ver si hay valores nulos "NULL"

sns.heatmap(seguros.isnull(), yticklabels = False, cbar = False, cmap="Blues")

In [ ]:
# Ver si hay valores nulos "NULL"

seguros.isnull().sum()

In [ ]:
seguros[['age', 'sex', 'bmi', 'children', 'smoker', 'charges']].hist(bins = 30, figsize = (12, 12), color = 'r');


In [ ]:
# realizar una gráfica pairplot

sns.pairplot(seguros)

In [ ]:
plt.figure(figsize = (15, 6))
sns.regplot(x = 'age', y = 'charges', data = seguros)
plt.show()
  

In [ ]:
plt.figure(figsize = (15, 6))
sns.regplot(x = 'bmi', y = 'charges', data = seguros)
plt.show()
  

**PRACTICA #3 [OPCIONAL]:**
 - **Calcule y grafique la matriz de correlación**
 - **¿Qué "feature" tiene la correlación más positiva con charges?**

## ENTRENAR MULTIPLES MODELOS UTILANDO AUTOGLUON

In [ ]:
# Dividir los datos en un 80% para entrenamiento y 20% para pruebas

from sklearn.model_selection import train_test_split
X_entreno, X_prueba = train_test_split(seguros, test_size=0.2, random_state=0)

In [ ]:
X_entreno

In [ ]:
X_prueba

Se entrenarán múltiples modelos ML de regresión utilizando AutoGluon

Es necesario especifcar la columna "etiqueta/meta/", datos de entreno, límite de tiempo, y pre-determinados

Nótese que AutoGluon detecta automáticamente si el problema es de clasificación o regresión basado en la columna "etiqueta"

Para problemas de tipo regresión, los valores de "etiqueta" son generalmente números de punto flotante con un gran número de valores únicos

In [ ]:
predictor = TabularPredictor(label="charges", 
                             problem_type = 'regression', 
                             eval_metric = 'r2').fit(train_data = X_entreno, 
                                                     time_limit = 200, 
                                                     presets = "best_quality")

In [ ]:
predictor.fit_summary()

## EVALUAR EL RENDIMIENTO DE LOS MODELOS ENTRENADOS

In [ ]:
predictor.leaderboard()

In [ ]:
# Inicializar la figura de matplotlib

f, ax = plt.subplots(figsize = (15, 6))
sns.barplot(x = "model", y = "score_val", data = predictor.leaderboard(), color = "b")
ax.set(ylabel = "Performance Metric (R2)", xlabel = "Regression Models")
plt.xticks(rotation = 45);

In [ ]:
predictor.evaluate(X_prueba)

In [ ]:
# Evaluar el rendimiento de los modelos
# Seleccionar 5 observaciones del conjunto de datos de prueba y generar las predicciones

y_pred = predictor.predict(X_prueba)
print("Predictions:  ", list(y_pred)[:5])

In [ ]:
X_prueba

In [ ]:
y_prueba = X_prueba['charges']
y_prueba # valores correctos

In [ ]:
y_predicc = predictor.predict(X_prueba)
plt.figure(figsize = (15, 10))
plt.plot(y_prueba, y_predicc, "^", color = 'r')
plt.ylabel('Predicciones de los modelos')
plt.xlabel('Valores reales')

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt

RMSE = float(format(np.sqrt(mean_squared_error(y_prueba, y_predicc)),'.3f'))
MSE = mean_squared_error(y_prueba, y_predicc)
MAE = mean_absolute_error(y_prueba, y_predicc)
r2 = r2_score(y_prueba, y_predicc)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2) 

**PRACTICA #4 [OPCIONAL]:**

- **Volver a en\ntrenar un modelo de regresión usando AutoGluon con valores pre-determinados diferentes**
- **Fijar el límite de tiempo a 300 secs**
- **Utilizar RMSE como la métrica principal y realice una gráfica de barras**
- **¿Qué modelo tiene el mejor rendimiento?**
- **Evaluar el rendimiento de los modelos entrenados por medio de la comparación de distintas métricas**

# ¡FABULOSO!